<a href="https://colab.research.google.com/github/iiojun/GDHP/blob/master/Hachioji_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

データをダウンロードする

In [ ]:
!wget https://www.iiojun.com/iDS/2023/hachimap.kml
!wget https://www.iiojun.com/iDS/2023/libraries.kml
!wget https://www.iiojun.com/iDS/2023/R04registers.xlsx


GeoPandas のインストール

In [ ]:
!pip install geopandas


モジュールの読み込み

In [ ]:
import geopandas as gpd
import fiona
import folium
import matplotlib.pyplot as plt
from fiona.drvsupport import supported_drivers


データの読み込み

In [ ]:
supported_drivers['KML'] = 'rw'
df = gpd.read_file('hachimap.kml', driver='KML')
df.head()
df

おためしプロット

In [ ]:
df.plot(figsize=(12,12))
plt.show()


地図の表示

In [ ]:
m = folium.Map(location=[35.66, 139.28], zoom_start=12, tiles='openstreetmap',
                zoom_control=False, scrollWheelZoom=False, dragging=False)
m


町丁目区画を重ねて表示する

In [ ]:
for _, r in df.iterrows():
  sim_geo = gpd.GeoSeries(r['geometry'])
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(data=geo_j,
        style_function=lambda x: {'fillColor': 'grey', 'color': 'grey',
          'weight': 0.5, 'fill_opacity': 0.3, 'line_opacity': 0.1 })
  folium.Popup(r['Name']).add_to(geo_j)
  geo_j.add_to(m)

m

図書館の位置を読み込む

In [ ]:
import pandas as pd
df_libs = gpd.read_file('libraries.kml', driver='KML')
df_libs


地図上に表示する

In [ ]:
for _, row in df_libs.iterrows():
  folium.Marker(location=[row['geometry'].y, row['geometry'].x], popup=row['Name']).add_to(m)
m

マーカーを小さいものにする．地図を描くところからやり直し

In [ ]:
m = folium.Map(location=[35.66, 139.30], zoom_start=12, tiles='openstreetmap',
                zoom_control=False, scrollWheelZoom=False, dragging=False)

for _, r in df.iterrows():
  sim_geo = gpd.GeoSeries(r['geometry'])
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(data=geo_j,
               style_function=lambda x: {'fillColor': 'grey', 'color': 'grey',
                                          'weight': 0.5, 'fill_opacity': 0.3,
                                          'line_opacity': 0.1 })
  folium.Popup(r['Name']).add_to(geo_j)
  geo_j.add_to(m)

for _, row in df_libs.iterrows():
  folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=4, color='red', fill_color='red', weight=3
  ).add_to(m)
m


R04registers.xlsx を読み込む

In [ ]:
df_reg = pd.read_excel('R04registers.xlsx')
df_reg

まず，区内だけ抽出

In [ ]:
df_reg = df_reg[df_reg['市内市外区分'] == '区内']
df_reg

おや？ 204 rows あるぞ？ 199 rows ちゃうんか？
抽出して比べてみる

In [ ]:
set1 = set(df['Name'])
set2 = set(df_reg['地区'])
questionnables = set2 - set1
questionnables

修正をかける．

堀之内一丁目, 鑓水一丁目, 高嶺町は単純に削除でOK

別所と南大沢は，それぞれ別所一丁目，南大沢一丁目に足し合わせる？

とりあえず全部ドロップしちゃうか……

In [ ]:
df_reg = df_reg[~df_reg['地区'].isin(questionnables)]
df_reg

ひとつの図書館だけ抜粋

In [ ]:
target = '南大沢'
df_l = df_reg[['地区', target]]
df_l = df_l.rename(columns = {'地区': 'Name', target: 'Users'})
df_l

In [ ]:
df1 = pd.merge(df.drop('Description', axis=1), df_l, on='Name')
df1

登録者数の最大値を求める

In [ ]:
max_users = max(df1['Users'])
max_users

df1のUsersを最大値で割って正規化する

In [ ]:
df1['Ratio'] = df1['Users'].apply(lambda x: 100 * float(x) / max_users)
df1

Ratioに応じた色を返す関数を定義する

In [ ]:
def rcolor(x): # x should be a value between 0 to 100
  n = int(2.55*x)
  if n > 255: n = 255
  return f'#{n:02x}4400'

スタイル関数を定義する

In [ ]:
def styles(x):
  c = x['properties']['color']
  return {'fillColor': c, 'color': c, 'fillOpacity': 0.5, 'opacity': 0.7, 'weight': 0.5 }

とりあえず地図に描いてみる

In [ ]:
import json

m = folium.Map(location=[35.66, 139.30], zoom_start=12, tiles='stamenterrain',
                zoom_control=False, scrollWheelZoom=False, dragging=False)
                # to disable zoom, scroll, and dragging functions

for _, r in df1.iterrows():
  sim_geo = gpd.GeoSeries(r['geometry'])
  geo_j = sim_geo.to_json()
  data = json.loads(geo_j)
  data['features'][0]['properties'] = {'color': rcolor(r['Ratio'])}
  geo_j = json.dumps(data)
  geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: styles(x))
  geo_j.add_to(m)

for _, row in df_libs.iterrows():
  c = 'red' if row['Description'] == target else 'yellow'
  folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=4, color=c, fill_color=c, weight=3
  ).add_to(m)
m

関数にまとめる

In [ ]:
def mk_map(target):
  df_l = df_reg[['地区', target]]
  df_l = df_l.rename(columns = {'地区': 'Name', target: 'Users'})
  df1 = pd.merge(df.drop('Description', axis=1), df_l, on='Name')
  max_users = max(df1['Users'])
  df1['Ratio'] = df1['Users'].apply(lambda x: 100 * float(x) / max_users)
  m = folium.Map(location=[35.66, 139.30], zoom_start=12, tiles='stamenterrain',
                zoom_control=False, scrollWheelZoom=False, dragging=False)

  for _, r in df1.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    data = json.loads(geo_j)
    data['features'][0]['properties'] = {'color': rcolor(r['Ratio'])}
    geo_j = json.dumps(data)
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: styles(x))
    geo_j.add_to(m)

  for _, row in df_libs.iterrows():
    c = 'red' if row['Description'] == target else 'yellow'
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=4, color=c, fill_color=c, weight=3
    ).add_to(m)

  m.save(f'map_{target}.html')

全ての図書館についての図を作る

In [ ]:
for d in df_libs['Description']:
  print(f'making a map: {d} ... ', end='')
  mk_map(d)
  print('done')

ダウンロード用にzipでまとめる

In [ ]:
!zip maps.zip map_*.html